# Demonstrate easy integration of the wandb client into pre-existing Beverage Packaging Waste dataset.

Reference: https://wandb.ai/sauravm/Optimizers/reports/How-to-Compare-Keras-Optimizers-in-Tensorflow-for-Deep-Learning--VmlldzoxNjU1OTA4



# Packages 📦 and Basic Setup
---



In [ ]:
%%capture

## Install the latest version of wandb client 🔥🔥
!pip install -q --upgrade wandb

## Import Packages
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
import os
import wandb
from wandb.keras import WandbMetricsLogger

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
wandb.init(project='Batch size')

# Feel free to change these and experiment !!
config = wandb.config
config.epochs = 30
config.batch_size = 32  # <-- Change this variable to experiment with various batch_size
config.img_height = 160 # <-- Change this variable to experiment with various image_size
config.img_width = 160  # <-- Change this variable to experiment with various image_size
config.validation_split = 0.2
config.seed = 42
config.optimizer = "adam" # <-- Change this variable to experiment with various optimizers

wandb.config.update(config)

# 💿 The Dataset
---

In [ ]:
from google.colab import drive #Connect to my Google Drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/rawimgs' #Directories of Google Drive
os.chdir(path) #Go to path
os.listdir(path) #List out files in the path
print("Files and directories in '", path, "' :", os.listdir(path))

'''
What datatypes are we looking at? What is the file structure?
'''

classes = sorted(os.listdir(path))
print(classes)

n_files = {}
filetypes = {}
for dirname, _, filenames in os.walk(path):
    n_files[dirname] = 0
    for filename in filenames:
        n_files[dirname] += 1   
        extension = filename.split('.')[-1]
        if extension in filetypes.keys():
            filetypes[extension] += 1
        else:    
            filetypes[extension] = 1

for directory, counts in n_files.items():
    print(f'number of files {directory} {counts}') 

for filetype, counts in filetypes.items():
    print(f'number of files of filetype {filetype} {counts}') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files and directories in ' /content/drive/My Drive/Colab Notebooks/rawimgs ' : ['AluCan', 'Glass', 'PET', 'HDPEM', 'Other Background ']
['AluCan', 'Glass', 'HDPEM', 'Other Background ', 'PET']
number of files /content/drive/My Drive/Colab Notebooks/rawimgs 0
number of files /content/drive/My Drive/Colab Notebooks/rawimgs/AluCan 1060
number of files /content/drive/My Drive/Colab Notebooks/rawimgs/Glass 1232
number of files /content/drive/My Drive/Colab Notebooks/rawimgs/PET 1508
number of files /content/drive/My Drive/Colab Notebooks/rawimgs/HDPEM 1028
number of files /content/drive/My Drive/Colab Notebooks/rawimgs/Other Background  1027
number of files of filetype jpg 4706
number of files of filetype JPG 1141
number of files of filetype HEIC 8


In [ ]:
import pathlib

data_dir = pathlib.Path(path)

image_count = len(list(data_dir.glob('*/*.jpg')))
wandb.config.update({"Image Count": image_count})

## Create tf.Dataset from utility function
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=config.validation_split,
  subset="training",
  seed=config.seed,
  image_size=(config.img_height, config.img_width),
  batch_size=config.batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=config.validation_split,
  subset="validation",
  seed=config.seed,
  image_size=(config.img_height, config.img_width),
  batch_size=config.batch_size)

## Add Class Names to the Config
class_names = train_ds.class_names
num_classes = len(class_names)
wandb.config.update({"Class Names": class_names})
wandb.config.update({"No of Classes": num_classes})

## Apply Optimizations
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Normalize the Dataset
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

Found 5847 files belonging to 5 classes.
Using 4678 files for training.
Found 5847 files belonging to 5 classes.
Using 1169 files for validation.


# ✍️ Model Architecture

---

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(config.img_height,
                                  config.img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

#🧱 + 🏗 = 🏠 Training

In [ ]:
model.compile(optimizer=config.optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=config.epochs,
  callbacks = [WandbMetricsLogger()])

wandb.finish()

Epoch 1/30


147/147 [==============================] - 513s 790ms/step - loss: 1.1054 - accuracy: 0.5372 - val_loss: 0.9987 - val_accuracy: 0.6168
Epoch 2/30
147/147 [==============================] - 23s 157ms/step - loss: 0.7106 - accuracy: 0.7277 - val_loss: 0.9994 - val_accuracy: 0.6364
Epoch 3/30
147/147 [==============================] - 21s 146ms/step - loss: 0.6038 - accuracy: 0.7708 - val_loss: 0.8238 - val_accuracy: 0.7169
Epoch 4/30
147/147 [==============================] - 23s 158ms/step - loss: 0.5343 - accuracy: 0.8029 - val_loss: 1.2785 - val_accuracy: 0.6407
Epoch 5/30
147/147 [==============================] - 21s 145ms/step - loss: 0.4845 - accuracy: 0.8181 - val_loss: 1.0355 - val_accuracy: 0.7015
Epoch 6/30
147/147 [==============================] - 23s 160ms/step - loss: 0.4321 - accuracy: 0.8405 - val_loss: 0.9420 - val_accuracy: 0.7297
Epoch 7/30
147/147 [==============================] - 23s 154ms/step - loss: 0.3795 - accuracy: 0.8598 - val_loss: 0.8283 - val_accuracy: 0.

epoch/accuracy,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇██████████████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▂▄▂▄▅▄▂▅▆▄▆▅▂▆▆▅▆▅▇▄▇█▆▆▅▅█▇▇
epoch/val_loss,▄▄▂▆▄▃▂▄▂▂▃▁▃█▂▃▃▃▃▂▇▃▁▄▅▅▆▂▂▂
epoch/accuracy,0.96986
epoch/epoch,29
epoch/learning_rate,0.001
epoch/loss,0.09121
epoch/val_accuracy,0.81608
